In [17]:
import pandas as pd

In [18]:
run = "out/2025-04-13_12-32-11_librispeech-pc-test-clean_cs16000_large-v3-turbo_ws15.0_ft6.csv"
df = pd.read_csv(run)
df

,id,wer,pred_transcription,time
0,2961-961-0000,0.272727,Socrates begins the Timaeus with a summary of ...,209.098996
1,4970-29093-0000,0.193333,You'll never dig it out of the Astor Library. ...,231.183140
2,6930-76324-0001,0.280093,there were certainly no near the solution of t...,160.168945
3,7729-102255-0000,0.285987,The Bogus legislature numbered 36 members. mem...,457.076795
4,5105-28240-0000,0.166667,Fast as his legs could carry him. Servodak had...,191.172782
...,...,...,...,...
78,1995-1837-0000,0.265347,"He knew the silver fleece, his and zora's must...",192.879359
79,237-126133-0000,0.278652,"Here she would stay, comforted and soothed amo...",175.684041
80,6829-68771-0000,0.280347,"So, to the surprise of the Democratic democrat...",252.892305
81,5683-32879-0000,0.243590,It was not very much past 11 eleven that morni...,198.427665


In [19]:
print("Average WER:", df["wer"].mean())
print("Standard deviation WER:", df["wer"].std())

Average WER: 0.24745206484293214
Standard deviation WER: 0.06618886462200975


In [20]:
def remove_pc(s: str) -> str:
    """Remove the PC from the string."""
    return s.lower().replace(".", "").replace(",", "").replace("'", "").replace('"', "").replace("`", "").replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace("{", "").replace("}", "").replace(";", "").replace(":", "").replace("!", "").replace("?", "")

In [21]:
def load_ground_truth(id: str) -> str:
    """Load the ground truth for the given id."""
    with open(f"data/librispeech-pc-test-clean/{id}/{id}.txt", "r") as f:
        return f.read().strip()

In [22]:
df["true_transcription"] = df["id"].apply(load_ground_truth)
df["true_transcription_clean"] = df["true_transcription"].apply(remove_pc)
df["pred_transcription_clean"] = df["pred_transcription"].apply(remove_pc)

In [23]:
import jiwer

df["wer_clean"] = df.apply(
    lambda row: jiwer.wer(row["true_transcription_clean"], row["pred_transcription_clean"]),
    axis=1,
)

In [24]:
librispeech_total_seconds = 17943.12
total_time = df["time"].sum()

In [27]:
print("Realtime factor", librispeech_total_seconds / total_time, "\n")
print("Mean WER:", df["wer"].mean())
print("Standard deviation WER:", df["wer"].std(), "\n")
print("Mean WER (no punctuation):", df["wer_clean"].mean())
print("Standard deviation WER (no punctuation):", df["wer_clean"].std())

Realtime factor 0.9235761415576417 

Mean WER: 0.24745206484293214
Standard deviation WER: 0.06618886462200975 

Mean WER (no punctuation): 0.12152826253174259
Standard deviation WER (no punctuation): 0.03893982084079961


In [26]:
# Remove true transcriptions from output
df = df.drop(columns=["true_transcription", "true_transcription_clean"])
df.to_csv(run, index=False)